In [ ]:
import yfinance as yf
import plotly.graph_objects as go
import numpy as np
import time

# Prompt the user to input the stock symbol
stock_symbol = input("Enter the stock symbol (e.g., BREN.JK): ")

# Function to fetch real-time stock price data
def get_realtime_data(symbol):
    try:
        stock_data = yf.Ticker(symbol)
        data = stock_data.history(period="1d", interval="1m")
        return data
    except Exception as e:
        print(f"An error occurred while fetching data: {e}")
        return None

# Function to update the candlestick chart
def update_candlestick_chart(fig, data):
    fig.add_trace(go.Candlestick(x=data.index,
                    open=data['Open'],
                    high=data['High'],
                    low=data['Low'],
                    close=data['Close']))

# Function to add a trend line to the chart
def add_trend_line(fig, data):
    x = np.arange(len(data))
    y = data['Close'].values

    # Calculate linear regression
    A = np.vstack([x, np.ones(len(x))]).T
    m, c = np.linalg.lstsq(A, y, rcond=None)[0]

    # Create trend line predictions
    trend_line = m * x + c

    fig.add_trace(go.Scatter(x=data.index, y=trend_line, mode='lines', line=dict(color='red', width=2), name='Trend Line'))

# Main program
if __name__ == "__main__":
    fig = go.Figure()

    while True:
        data = get_realtime_data(stock_symbol)

        if data is not None and not data.empty:
            # Update the candlestick chart
            update_candlestick_chart(fig, data)
            add_trend_line(fig, data)
            fig.update_layout(title=f"Candlestick Chart with Trend Line for {stock_symbol}",
                              xaxis_title="Time",
                              yaxis_title="Price")
            fig.show()

        # Wait for 5 minute before fetching the next data
        time.sleep(300)
